---NHIỆM VỤ 1---

In [13]:
import pandas as pd
import numpy as np
owner_df = pd.read_csv('owner_info.csv',encoding='utf-8')
registration_df = pd.read_csv('vehicle_registration.csv',encoding='utf-8')
emission_df = pd.read_csv('emission_test.csv',encoding='utf-8')

---OWNER---

In [14]:
owner_df['owner_id']=owner_df['owner_id'].astype(str).str.strip().str.upper()
owner_df['owner_name']=owner_df['owner_name'].astype(str).str.strip().str.title().str.replace(r'\s+', ' ', regex=True)
owner_df['province']=owner_df['province'].astype(str).str.strip().str.title().str.replace(r'\+', '', regex=True)
sua_ten ={
    'Tp Hcm': 'TP.HCM', 'Tp.Hcm': 'TP.HCM', 'Tp. Hcm': 'TP.HCM',
    'Ha Noi': 'Hà Nội', 'Hanoi': 'Hà Nội',
    'Da Nang': 'Đà Nẵng', 'Danang': 'Đà Nẵng',
    'Binh Duong': 'Bình Dương'
}
owner_df['province'] = owner_df['province'].replace(sua_ten)
owner_df['province'] = owner_df['province'].replace('Binh  Duong', 'Bình Dương')

---REGISTRATION---

In [15]:
registration_df['plate_no']=registration_df['plate_no'].astype(str).str.strip().str.replace(' ', '')
registration_df['owner_id'] = registration_df['owner_id'].astype(str).str.strip().str.upper()
registration_df['vehicle_type']= registration_df['vehicle_type'].astype(str).str.lower().str.strip().str.replace(r'\s+', ' ', regex=True)
dang_xe = {
    'xe may': 'Xe máy', 'xemay': 'Xe máy', 'xe máy': 'Xe máy',
    'o to con': 'Ô tô con', 'oto con': 'Ô tô con', 'ô tô con': 'Ô tô con', 'ô tô': 'Ô tô con', 'o to': 'Ô tô con',
    'xe tai': 'Xe tải', 'xetai': 'Xe tải', 'xe tải': 'Xe tải'
}
registration_df['vehicle_type'] = registration_df['vehicle_type'].replace(dang_xe)
registration_df['engine_cc'] = registration_df['engine_cc'].astype(str).str.lower().str.replace('cc', '').str.strip()
registration_df['engine_cc'] = pd.to_numeric(registration_df['engine_cc'], errors='coerce').abs()
registration_df['reg_date'] = pd.to_datetime( registration_df['reg_date'], format='mixed',dayfirst=True,errors='coerce')

---EMISSON---

In [16]:
emission_df['plate_no'] = emission_df['plate_no'].astype(str).str.strip().str.replace(' ', '')
emission_df['test_date'] = pd.to_datetime(emission_df['test_date'].astype(str).str.strip(), dayfirst=True,  format='mixed', errors='coerce')
def ket_qua(gia_tri):
    gt = str(gia_tri).lower().strip()
    if pd.isna(gia_tri): 
        return np.nan
    if 'k' in gt:
        return 'Không đạt'
    if 'dat' in gt or 'đạt' in gt:
        return 'Đạt'
    return 'không đạt'
emission_df['result'] = emission_df['result'].apply(ket_qua)
emission_df['co_level'] = emission_df['co_level'].astype(str).str.replace(',', '.').str.replace('%', '').str.strip()
emission_df['co_level'] = pd.to_numeric(emission_df['co_level'], errors='coerce')
emission_df.loc[emission_df['co_level'] < 0, 'co_level'] = np.nan
emission_df['co_level'] = emission_df['co_level'].fillna(emission_df['co_level'].median())
owner_df.to_csv('owner_info_sach.csv', index=False, encoding='utf-8-sig')
registration_df.to_csv('vehicle_registration_sach.csv', index=False, encoding='utf-8-sig')
emission_df.to_csv('emission_test_sach.csv', index=False, encoding='utf-8-sig')

---NHIỆM VỤ 2---

In [17]:
df_owner = pd.read_csv('owner_info_sach.csv')
df_vehicle = pd.read_csv('vehicle_registration_sach.csv')
df_emission = pd.read_csv('emission_test_sach.csv')
print('1.số lượng chủ xe theo tỉnh:')
print(df_owner['province'].value_counts())
print('2.số lượng xe theo loại xe:')
print(df_vehicle['vehicle_type'].value_counts())
df_vehicle['reg_date'] = pd.to_datetime(df_vehicle['reg_date'])
print('3.số lượng đăng ký theo năm:')
so_luong_theo_nam = df_vehicle['reg_date'].dt.year.value_counts().sort_index()
print(so_luong_theo_nam)
print('4.số lượng xe không có thông tin kiểm định khí thải:')
mask_miss = ~df_vehicle['plate_no'].isin(df_emission['plate_no'])
xe_chua_kiem_dinh= df_vehicle[mask_miss]
print(f"5.Tổng số xe chưa kiểm định: {len(xe_chua_kiem_dinh)}")
ty_le= pd.merge(df_emission, df_vehicle[['plate_no', 'vehicle_type']], on='plate_no', how='inner')
ty_le['fail'] = ty_le['result'].apply(lambda x: 1 if x == 'Không đạt' else 0)
fail=ty_le.groupby('vehicle_type')['fail'].mean() * 100
fail=fail.sort_values(ascending=False)
print(fail.round(2).astype(str) + '%')


1.số lượng chủ xe theo tỉnh:
province
Đà Nẵng       17
Hà Nội        16
Bình Dương    15
TP.HCM        12
Name: count, dtype: int64
2.số lượng xe theo loại xe:
vehicle_type
Xe máy      47
Xe tải      39
Ô tô con    34
Name: count, dtype: int64
3.số lượng đăng ký theo năm:
reg_date
2021    40
2022    39
2023    41
Name: count, dtype: int64
4.số lượng xe không có thông tin kiểm định khí thải:
5.Tổng số xe chưa kiểm định: 57
vehicle_type
Xe tải      57.14%
Ô tô con    52.38%
Xe máy      41.46%
Name: fail, dtype: object
